In [ ]:
import re

import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
with open("pid_table.html", "r") as f:
    soup = BeautifulSoup(f.read(), "html.parser")
table = soup.find("table")

In [ ]:
table_header = table.find("thead")
col_headers = [el.text.strip() for el in table_header.find_all("th")]
col_headers[col_headers.index("Data bytes\n                    returned")] = (
    "Data bytes returned"
)

In [ ]:
data = []
table_body = table.find("tbody")
rows = table_body.find_all("tr")
for row in rows:
    cols = row.find_all("td")
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])  # Get rid of empty values

In [ ]:
df = pd.DataFrame(data, columns=col_headers)
df = df.fillna("")
df = df.rename(
    columns={
        "Formula[a]": "Formula",
        "Data bytes returned": "Data bytes",
        "Min value": "Min",
        "Max value": "Max",
        "PID(Dec)": "PID",
    }
)
df = df.drop("PIDs(hex)", axis=1)
for col in df.columns:
    df[col] = df[col].apply(
        lambda x: " ".join(filter(len, map(str.strip, x.split("\n"))))
    )
df["Formula_raw"] = df["Formula"]
for col in ["Formula"]:
    if col.endswith("_raw"):
        continue
    df[col] = (
        df[col]
        .apply(lambda x: re.sub(r"\\t?frac {([^}]+)}{([^}]+)}", r"(\1/\2)", x))
        .apply(lambda x: re.sub(r"\\displaystyle {([^}]+)}", r"\1", x))
        .apply(lambda x: re.sub(r"{\\displaystyle ([^}]+)}", r"(\1)", x))
        .apply(lambda x: re.sub(r"{|}", r"", x))
    )
df.to_csv("../pid_table.csv", index=False)
df